In [2]:
import rioxarray
import geopandas as gpd

ndvi = rioxarray.open_rasterio("NDVI.tif")
ndvi_sq = ndvi.squeeze()

field = gpd.read_file("cropped_field.shp")
field.crs

<Derived Projected CRS: EPSG:28992>
Name: Amersfoort / RD New
Axis Info [cartesian]:
- X[east]: Easting (metre)
- Y[north]: Northing (metre)
Area of Use:
- name: Netherlands - onshore, including Waddenzee, Dutch Wadden Islands and 12-mile offshore coastal zone.
- bounds: (3.2, 50.75, 7.22, 53.7)
Coordinate Operation:
- name: RD New
- method: Oblique Stereographic
Datum: Amersfoort
- Ellipsoid: Bessel 1841
- Prime Meridian: Greenwich

In [3]:
field_to_raster_crs = field.to_crs(ndvi.rio.crs)
field_to_raster_crs

,category,gewas,gewascode,jaar,status,geometry
0,Grasland,"Grasland, blijvend",265,2020,Definitief,"POLYGON ((627394.386 5812746.538, 627393.681 5..."
1,Grasland,"Grasland, blijvend",265,2020,Definitief,"POLYGON ((627326.170 5813192.913, 627324.734 5..."
2,Grasland,"Grasland, blijvend",265,2020,Definitief,"POLYGON ((627093.646 5812863.661, 627090.972 5..."
3,Grasland,"Grasland, blijvend",265,2020,Definitief,"POLYGON ((627384.004 5813467.330, 627376.549 5..."
4,Grasland,"Grasland, natuurlijk. Hoofdfunctie landbouw.",331,2020,Definitief,"POLYGON ((627179.293 5812870.376, 627160.747 5..."
...,...,...,...,...,...,...
4867,Grasland,"Grasland, natuurlijk. Hoofdfunctie landbouw.",331,2020,Definitief,"POLYGON ((640710.959 5809885.718, 640703.030 5..."
4868,Grasland,"Grasland, natuurlijk. Hoofdfunctie landbouw.",331,2020,Definitief,"POLYGON ((640753.587 5810143.151, 640751.454 5..."
4869,Grasland,"Grasland, blijvend",265,2020,Definitief,"POLYGON ((640890.832 5809998.743, 640892.271 5..."
4870,Grasland,"Grasland, natuurlijk. Hoofdfunctie landbouw.",331,2020,Definitief,"POLYGON ((640770.878 5810005.390, 640766.767 5..."


In [4]:
geom = field_to_raster_crs[["geometry", "gewascode"]].values.tolist()
geom

[[<POLYGON ((627394.386 5812746.538, 627393.681 5812749.022, 627384.063 581279...>,
  265],
 [<POLYGON ((627326.17 5813192.913, 627324.734 5813196.828, 627324.11 5813203....>,
  265],
 [<POLYGON ((627093.646 5812863.661, 627090.972 5812870.702, 627076.762 581286...>,
  265],
 [<POLYGON ((627384.004 5813467.33, 627376.549 5813470.117, 627385.233 5813495...>,
  265],
 [<POLYGON ((627179.293 5812870.376, 627160.747 5812846.868, 627159.505 581284...>,
  331],
 [<POLYGON ((627419.816 5812765.254, 627416.356 5812766.811, 627413.1 5812769....>,
  265],
 [<POLYGON ((627358.683 5813335.376, 627362.871 5813352.853, 627370.175 581337...>,
  265],
 [<POLYGON ((627423.06 5813550.358, 627421.409 5813550.392, 627427.293 5813568...>,
  265],
 [<POLYGON ((627191.219 5813028.547, 627189.596 5813028.866, 627186.64 5813029...>,
  265],
 [<POLYGON ((627349.601 5813329.836, 627343.37 5813330.159, 627341.775 5813330...>,
  265],
 [<POLYGON ((627809.311 5811941.534, 627793.366 5811967.477, 627778.317 581199..

In [6]:
from rasterio import features

field_cropped_raster = features.rasterize(geom, out_shape=ndvi_sq.shape, fill=0, transform=ndvi.rio.transform())

In [7]:
import xarray as xr
field_cropped_raster_xarr = xr.DataArray(field_cropped_raster)


In [8]:
from xrspatial import zonal_stats

zonal_stats(field_cropped_raster_xarr, ndvi_sq)

,zone,mean,max,min,sum,std,var,count
0,0,0.266528,0.999579,-0.998648,38887.554688,0.409970,0.168075,145904.0
1,259,0.520282,0.885242,0.289196,449.003052,0.111205,0.012366,863.0
2,265,0.775609,0.925955,0.060755,66478.976562,0.091089,0.008297,85712.0
3,266,0.794128,0.918048,0.544686,1037.925781,0.074009,0.005477,1307.0
4,331,0.703056,0.905304,0.142226,10725.819336,0.102255,0.010456,15256.0
5,332,0.681699,0.849158,0.178113,321.080261,0.123633,0.015285,471.0
6,335,0.648063,0.865804,0.239661,313.662598,0.146582,0.021486,484.0
7,863,0.388575,0.510572,0.185987,1.165724,0.144245,0.020807,3.0


In [10]:
# Challenge: Calculate zonal statistics for zones defined by ndvi_classified
ndvi = rioxarray.open_rasterio("NDVI.tif")
ndvi_classified = rioxarray.open_rasterio("NDVI_classified.tif")
ndvi_sq = ndvi.squeeze()
ndvi_classified_sq = ndvi_classified.squeeze()

zonal_stats(ndvi_classified_sq, ndvi_sq)

,zone,mean,max,min,sum,std,var,count
0,1,-0.355660,-0.000257,-0.998648,-12838.253906,0.145916,0.021291,36097.0
1,2,0.110731,0.199839,0.000000,1754.752441,0.055864,0.003121,15847.0
2,3,0.507998,0.700000,0.200000,50410.167969,0.140193,0.019654,99233.0
3,4,0.798281,0.999579,0.700025,78888.523438,0.051730,0.002676,98823.0
